In [4]:
import cv2
import numpy as np
import os
from random import shuffle
import tqdm

TRAIN_DIR = 'Desktop/train'
TEST_DIR = 'Desktop/test'
IMG_SIZE = 50
LR = 0.0001

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '6conv-basic')


In [5]:
def label_image(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat' : return [1,0]
    elif word_label == 'dog' : return [0,1]

In [6]:
def create_train_data():
    training_data = []
    for img in os.listdir(TRAIN_DIR):
        label = label_image(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data
        
        
        
        

In [7]:
def process_test_data():
    testing_data = []
    for img in os.listdir(TEST_DIR):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    np.save('test_data.npy', testing_data)
    return testing_data

        
        

In [8]:
train_data = create_train_data()

In [9]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None,IMG_SIZE,IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet,2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate = LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir = 'log')


C:\Users\Dell\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print("Model Loaded!")

INFO:tensorflow:Restoring parameters from C:\Users\Dell\dogsvscats-0.0001-6conv-basic.model
Model Loaded!


In [11]:
train = train_data[:-500]
test = train_data[-500:]

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id='MODEL_NAME')

Training Step: 2965  | total loss: 0.45228 | time: 96.443s
| Adam | epoch: 003 | loss: 0.45228 - acc: 0.7853 -- iter: 18176/24500


In [ ]:
model.save(MODEL_NAME)

In [ ]:
import matplotlib.pyplot as plt
test_data = process_test_data()
fig = plt.figure()

for num,data in enumerate(test_data[:12]):
    img_data = data[0]
    img_num = data[1]
    y = fig.add_subplot(3,4, num + 1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: str_label = 'Dog'
    else: str_label = 'Cat'
    
    y = imshow(orig, cmap = 'gray')
    plt.title("str_label")
    
    y.axes.get_xaxes().set_visible(False)
    y.axes.get_yaxes().set_visible(False)
    
plt.show()


In [ ]:
with open('submission-file.csv','w') as f:
    f.write('id,label\n')
    

In [ ]:
with open('submission-file.csv','a') as f:
    for data in tqdm(test_data):
        img_data = data[0]
        img_num = data[1]
        orig = img_data
        data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        model_out = model.predict([data])[0]
        f.write('{},{}\n'.format(img_num, model_out[1]))